# Example - Zonal Statistics
This is useful in the case where you want to get regional statistics for a raster.

In [1]:
import geopandas as gpd
import numpy
import rioxarray
import xarray

from geocube.api.core import make_geocube

## Load in the source raster data

In [2]:
#!wget https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/IMG/USGS_NED_13_n42w091_IMG.zip

In [3]:
elevation = rioxarray.open_rasterio(
    "zip://USGS_NED_13_n42w091_IMG.zip/USGS_NED_13_n42w091_IMG.img"
).squeeze().drop("band")
elevation.name = "elevation"

## Create the data mask by rasterizing the unique ID of the vector data

In [4]:
ssurgo_data = gpd.read_file("../../test/test_data/input/soil_data_group.geojson")
ssurgo_data = ssurgo_data.loc[ssurgo_data.hzdept_r==0]
# convert the key to group to the vector data to an integer as that is one of the
# best data types for this type of mapping. If your data is not integer,
# then consider using a mapping of your data to an integer with something
# like a categorical dtype.
ssurgo_data["mukey"] = ssurgo_data.mukey.astype(int)

In [5]:
out_grid = make_geocube(
    vector_data=ssurgo_data,
    measurements=["mukey"],
    like=elevation, # ensure the data are on the same grid
    fill=numpy.nan
)

In [6]:
# merge the two together
out_grid["elevation"] = elevation
out_grid

<xarray.Dataset>
Dimensions:      (x: 10812, y: 10812)
Coordinates:
  * y            (y) float64 42.0 42.0 42.0 42.0 42.0 ... 41.0 41.0 41.0 41.0
  * x            (x) float64 -91.0 -91.0 -91.0 -91.0 ... -90.0 -90.0 -90.0 -90.0
    spatial_ref  int64 0
Data variables:
    mukey        (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    elevation    (y, x) float32 ...
Attributes:
    grid_mapping:  spatial_ref

### Get the elevation statistics of each region using the mask

In [7]:
grouped_elevation = out_grid.drop("spatial_ref").groupby(out_grid.mukey)
grid_mean = grouped_elevation.mean().rename({"elevation": "elevation_mean"})
grid_min = grouped_elevation.min().rename({"elevation": "elevation_min"})
grid_max = grouped_elevation.max().rename({"elevation": "elevation_max"})
grid_std = grouped_elevation.std().rename({"elevation": "elevation_std"})

In [8]:
zonal_stats = xarray.merge([grid_mean, grid_min, grid_max, grid_std])
zonal_stats.to_dataframe()

,elevation_mean,elevation_min,elevation_max,elevation_std
mukey,,,,
198692.0,173.130920,169.394562,188.442505,3.307044
198714.0,175.061554,170.214157,179.716675,2.148920
198724.0,179.933060,178.237244,181.490387,0.628017
198750.0,176.188461,167.951233,190.138763,3.814724
198754.0,171.632187,167.610321,181.611298,2.996241
271425.0,167.973709,167.951233,168.476776,0.076759
271431.0,176.718338,170.258133,180.460220,2.732229
